## Observations:
zip is available in address - index on split is -1, -5:

potential interests:
- zip code (reduced to large(-ish?) sample sizes)
- maybe look at geographical location specifically? (Is proximity to city center and features calculable?| $\checkmark$)
  - look into part 2 ^
  - waterfront is listed as a binary column
- (person pref, focus on home buyers/individuals)
- someone mentioned school data - this isn't a bad idea.
  - can get school rankings and their zip codes

would like to get external/additional information. Will check Kaggle for related data sets

https://kingcounty.gov/ <= city website

find out if prices are based on closes or valuations - if closes, can 

### Data cleaning
- multiple y/n / cat. columns that can be converted to dummies for OLS discovery
- view quality can be rated as good/bad/avg. , could be extrapolated to 0-2 (more if more options there)
- grade?

<hr/>
Data from [column_names.md](data/column_names.md)  
vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv

* `view` - Quality of view from house
  * Includes views of Mt. Rainier, Olympics, Cascades, Territorial, Seattle Skyline, Puget Sound, Lake Washington, Lake Sammamish, small lake / river / creek, and other
* `condition` - How good the overall condition of the house is. Related to maintenance of house.
  * See the [King County Assessor Website](https://info.kingcounty.gov/assessor/esales/Glossary.aspx?type=r) for further explanation of each condition code
* `grade` - Overall grade of the house. Related to the construction and design of the house.
  * See the [King County Assessor Website](https://info.kingcounty.gov/assessor/esales/Glossary.aspx?type=r) for further explanation of each building grade code
<hr/>

lots of relevant info for discovery in there

# _*IMPORTANT*_

Most fields were pulled from the [King County Assessor Data Download](https://info.kingcounty.gov/assessor/DataDownload/default.aspx).

The `address`, `lat`, and `long` fields have been retrieved using a third-party [geocoding API](https://docs.mapbox.com/api/search/geocoding/). In some cases due to missing or incorrectly-entered data from the King County Assessor, this API returned locations outside of King County, WA. If you plan to use the `address`, `lat`, or `long` fields in your modeling, consider identifying outliers prior to including the values in your model

<br/><hr/><br/><hr/>

### imports

In [42]:
import pandas as pd

In [2]:
housing_df = pd.read_csv('data/kc_house_data.csv')

<hr/>

## get_zip(s)() block

In [3]:
# TODO: ##################################
# move this to code/ if it proves useful #
##########################################

import geopy # <= `conda install -c conda-forge geopy`

def get_zip(
    latitude,
    longitude,
    return_errors=False
    ):
    """_summary_

    Args:
        latitude (float): the latitude of a target address
        longitude (float): the longitude of a target address
        return_errors (bool): whether to return errors or None, useful for "dirty" data. Defaults to False.

    Returns:
        str: string zip/post code of the requested long/lat
    """
    geoloc = geopy.Nominatim(user_agent='check_1')
    try:
        location = geoloc.reverse(f"{latitude},{longitude}")
        return location.raw['address']['postcode']
    except:
        return ValueError() if return_errors else None

def get_zips(
    latitudes,
    longitudes,
    return_errors=False
    ):
    """_summary_

    Args:
        latitude (list): the latitudes of a target address. Should be a list of floats.
        longitude (list): the longitudes of a target address. Should be a list of floats.
        return_errors (bool, optional): whether to return errors or None, useful for "dirty" data. Defaults to False.

    Returns:
        list: list of zip codes (str) based on latitude and longitude provided
    """
    try:
        if len(latitudes) == len(longitudes):
            location = [get_zip(_lat, _long, return_errors) for (_lat,_long) in zip(latitudes,longitudes)]
            return location
    except:
        if return_errors:
            raise ValueError(f'Error(s) raised in thread: {Exception}')
        else:
            return None

In [4]:
# display(
#     housing_df[['lat','long']],
#     housing_df[['lat','long']].info()
#     )

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30155 entries, 0 to 30154
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   lat     30155 non-null  float64
 1   long    30155 non-null  float64
dtypes: float64(2)
memory usage: 471.3 KB


,lat,long
0,47.461975,-122.19052
1,47.711525,-122.35591
2,47.502045,-122.22520
3,47.566110,-122.29020
4,47.532470,-122.07188
...,...,...
30150,47.664740,-122.32940
30151,47.565610,-122.38851
30152,47.610395,-122.29585
30153,47.449490,-122.18908


None

In [5]:
# get_zip(47.449490, -122.18908)

'98055'

In [6]:
# get_zips(
#     list(housing_df['lat'])[:10],
#     list(housing_df['long'])[:10],
#     True
#     )

['98055',
 '98133',
 '98178',
 '98118',
 '98027',
 '98155',
 '98166',
 '98030',
 '98023',
 '98019']

In [43]:
# ofst = 150
# z = ofst + 0
# slc = ofst + 100
# get_zips(
#     list(housing_df['lat'])[z:slc],
#     list(housing_df['long'])[z:slc],
#     return_errors = False
#     )

<hr/>

In [11]:
# This is 1,000,000x more efficient than re-calling the geopy functions since this is available.

housing_df['zip2'] = housing_df['address'].map(lambda x: x.split(',')[-2][-5:])

In [12]:
housing_df.iloc[:,-4:].head()

,address,lat,long,zip2
0,"2102 Southeast 21st Court, Renton, Washington ...",47.461975,-122.19052,98055
1,"11231 Greenwood Avenue North, Seattle, Washing...",47.711525,-122.35591,98133
2,"8504 South 113th Street, Seattle, Washington 9...",47.502045,-122.22520,98178
3,"4079 Letitia Avenue South, Seattle, Washington...",47.566110,-122.29020,98118
4,"2193 Northwest Talus Drive, Issaquah, Washingt...",47.532470,-122.07188,98027


In [31]:
housing_df['zip2'].value_counts()

98042    992
98038    858
98103    761
98115    761
98117    748
        ... 
62401      1
55411      1
58042      1
55369      1
94607      1
Name: zip2, Length: 399, dtype: int64

In [36]:
# get zip codes that have at least 100 entries

zip_counts = housing_df['zip2'].value_counts().rename('zip_counts')

In [39]:
zips_counted = housing_df.merge(
    zip_counts,
    left_on='zip2',
    right_index=True
)

In [40]:
zips_counted.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30155 entries, 0 to 29900
Data columns (total 27 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             30155 non-null  int64  
 1   date           30155 non-null  object 
 2   price          30155 non-null  float64
 3   bedrooms       30155 non-null  int64  
 4   bathrooms      30155 non-null  float64
 5   sqft_living    30155 non-null  int64  
 6   sqft_lot       30155 non-null  int64  
 7   floors         30155 non-null  float64
 8   waterfront     30155 non-null  object 
 9   greenbelt      30155 non-null  object 
 10  nuisance       30155 non-null  object 
 11  view           30155 non-null  object 
 12  condition      30155 non-null  object 
 13  grade          30155 non-null  object 
 14  heat_source    30123 non-null  object 
 15  sewer_system   30141 non-null  object 
 16  sqft_above     30155 non-null  int64  
 17  sqft_basement  30155 non-null  int64  
 18  sqft_g

In [41]:
zips_counted[zips_counted['zip_counts'] >= 100]

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,greenbelt,...,sqft_basement,sqft_garage,sqft_patio,yr_built,yr_renovated,address,lat,long,zip2,zip_counts
0,7399300360,5/24/2022,675000.0,4,1.0,1180,7140,1.0,NO,NO,...,0,0,40,1969,0,"2102 Southeast 21st Court, Renton, Washington ...",47.461975,-122.190520,98055,219
40,3340401570,3/2/2022,750000.0,3,2.0,1830,7969,1.0,NO,NO,...,930,240,90,1950,2008,"306 South 15th Street, Renton, Washington 9805...",47.466730,-122.214000,98055,219
111,7399301200,3/29/2022,728000.0,4,2.0,2170,7520,1.0,NO,NO,...,1240,490,60,1973,0,"1814 Aberdeen Avenue Southeast, Renton, Washin...",47.463930,-122.189740,98055,219
229,9899200050,3/24/2022,565000.0,4,2.0,1400,10364,1.5,NO,NO,...,0,330,330,1971,0,"3426 Shattuck Avenue South, Renton, Washington...",47.448450,-122.212430,98055,219
308,6673070070,12/28/2021,645000.0,3,2.0,1520,8250,1.0,NO,NO,...,590,420,200,1981,0,"2209 Southeast 21st Street, Renton, Washington...",47.460870,-122.188690,98055,219
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29809,723039050,12/16/2021,1826000.0,2,3.0,2070,71264,2.0,NO,NO,...,1360,480,952,2001,0,"10717 Southwest Biloxi Road, Vashon, Washingto...",47.504070,-122.471550,98070,195
29842,2022039037,4/14/2022,256000.0,2,1.0,1330,113815,1.0,NO,NO,...,0,280,0,1911,1962,"9632 Southwest Bayview Drive, Vashon, Washingt...",47.387960,-122.456440,98070,195
29913,2322039033,7/14/2021,950000.0,2,2.5,1960,11406,2.0,NO,NO,...,840,420,560,1938,1985,"4510 Southwest 244th Street, Vashon, Washingto...",47.386190,-122.387935,98070,195
29985,3023039221,9/21/2021,450000.0,2,2.0,840,138520,1.0,NO,NO,...,0,0,230,2020,0,"16097 109th Avenue Southwest, Vashon, Washingt...",47.458555,-122.475450,98070,195
